In [97]:
import warnings
warnings.filterwarnings("ignore")
from sqlalchemy import create_engine
from  datetime import datetime,timedelta
import pandas as pd
import fast_sql
pd.set_option('display.max_rows', 400)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [98]:
stock = pd.read_csv(r"D:\work_dir\stock\src\stockProject\get_ma_A_ma30_stock.csv")
stock.jx_date = stock.jx_date.astype('datetime64')
stock.symbol = stock.symbol.apply(lambda x:str(x).zfill(6))
stock = stock.loc[(stock['var']<=0.014) & (stock.pe<=65)&(stock.jx_counts>=2)]

In [99]:
origin =  pd.read_pickle(r"D:\work_dir\stock\src\stockProject\history_data\stock_2020-11-09.pkl")
origin.date = origin.date.astype('datetime64')
origin = origin.sort_values(['symbol','date']).reset_index(drop=True)
origin['af_colse_15'] = -origin.groupby('symbol').apply(lambda x: x['close'].diff(-12) / x['close'] ).\
                          reset_index().sort_values('level_1')['close']

In [100]:
x = stock.query("stock_name=='完美世界'")

In [101]:
%%time
pord_dict = {}
for symbol,df in x.groupby(['symbol']):
    now = datetime.today() - timedelta(days=12)
    date_list = {}
    for date in df.jx_date:
        month = date.strftime('%Y%m')
        if month not in date_list and date<now:
            date_list[month] = date

    date_list = [date_list[i] for i in date_list]
    origin_df = origin[origin.symbol==symbol].sort_values('date').set_index('date',drop=False)
    success = origin_df[origin_df.date.isin(date_list)].af_colse_15>0.1
    print(success,success.sum())
    pord = success.sum()/ len(success)
    pord_dict[symbol] = pord
x['pord'] = x.symbol.apply(lambda x:pord_dict[x])

date
2019-08-08     True
2019-09-03    False
2019-10-08    False
2019-11-01     True
2020-01-23    False
2020-02-03    False
2020-03-02    False
2020-04-02    False
2020-06-05     True
2020-08-31    False
2020-09-01    False
2020-10-09    False
Name: af_colse_15, dtype: bool 3
Wall time: 163 ms


In [103]:
x

,jx_date,var,ma5,ma10,ma20,ma30,jx_counts,jx_names,symbol,t_date,open_price,high,close_price,low,volume,price_change,p_change,v_ma5,v_ma10,v_ma20,turnover,stock_name,industry,pe,bf_close,af_close,sw_class,pord_5,pord_10,pord_15,pord_30,pord
0,2019-08-08,0.012239,-0.03,0.16,0.05,-0.13,2,"-0.03,0.16,0.05,-0.13",002624,2019-08-08,25.09,25.54,24.81,24.62,68591.96,-0.19,-0.76,91399.18,90866.38,73665.58,0.59,完美世界,影视音像,24.23,NaN,0.14,get_ma_A_ma30,0.416667,0.25,0.272727,0.222222,0.25
1,2019-08-09,0.011243,-0.10,0.13,0.04,-0.15,2,"-0.1,0.13,0.04,-0.15",002624,2019-08-09,24.95,25.04,24.24,24.00,99209.42,-0.57,-2.30,91751.62,95562.66,76210.49,0.86,完美世界,影视音像,24.23,NaN,0.16,get_ma_A_ma30,0.416667,0.25,0.272727,0.222222,0.25
2,2019-08-12,0.011672,-0.19,0.09,0.05,-0.15,2,"-0.19,0.09,0.05,-0.15",002624,2019-08-12,24.65,26.02,25.95,24.53,122288.57,1.71,7.05,98140.03,100051.80,78365.34,1.05,完美世界,影视音像,24.23,-0.01,0.11,get_ma_A_ma30,0.416667,0.25,0.272727,0.222222,0.25
3,2019-08-13,0.010415,-0.19,0.06,0.05,-0.11,2,"-0.19,0.06,0.05,-0.11",002624,2019-08-13,25.93,26.38,26.24,25.56,77901.73,0.29,1.12,92662.97,94145.25,79364.80,0.67,完美世界,影视音像,24.23,-0.00,0.12,get_ma_A_ma30,0.416667,0.25,0.272727,0.222222,0.25
4,2019-08-14,0.008033,-0.11,0.03,0.06,-0.06,2,"-0.11,0.03,0.06,-0.06",002624,2019-08-14,26.80,27.37,26.27,26.24,203142.52,0.03,0.11,114226.84,104543.70,84104.70,1.75,完美世界,影视音像,24.23,0.02,0.11,get_ma_A_ma30,0.416667,0.25,0.272727,0.222222,0.25
5,2019-08-15,0.008170,0.01,0.03,0.09,0.00,4,"0.01,0.03,0.09,0.0",002624,2019-08-15,25.99,27.09,26.84,25.82,149842.19,0.57,2.17,130476.89,110938.03,88718.93,1.29,完美世界,影视音像,24.23,0.05,0.05,get_ma_A_ma30,0.416667,0.25,0.272727,0.222222,0.25
6,2019-08-16,0.012601,0.19,0.04,0.11,0.06,4,"0.19,0.04,0.11,0.06",002624,2019-08-16,26.75,28.82,27.82,26.62,220101.22,0.98,3.65,154655.25,123203.43,97480.21,1.90,完美世界,影视音像,24.23,0.11,0.02,get_ma_A_ma30,0.416667,0.25,0.272727,0.222222,0.25
7,2019-09-03,0.012153,-0.24,0.02,0.07,0.03,3,"-0.24,0.02,0.07,0.03",002624,2019-09-03,27.56,28.19,27.89,27.46,115336.05,0.31,1.12,102274.49,111369.40,130208.79,0.99,完美世界,影视音像,24.23,-0.04,0.06,get_ma_A_ma30,0.416667,0.25,0.272727,0.222222,0.25
8,2019-09-04,0.011413,-0.20,-0.02,0.08,0.03,2,"-0.2,-0.02,0.08,0.03",002624,2019-09-04,27.99,28.75,28.58,27.62,118073.41,0.69,2.47,103127.79,101750.75,131346.30,1.02,完美世界,影视音像,24.23,0.01,0.04,get_ma_A_ma30,0.416667,0.25,0.272727,0.222222,0.25
9,2019-09-05,0.010316,-0.10,-0.04,0.11,0.03,2,"-0.1,-0.04,0.11,0.03",002624,2019-09-05,28.24,29.29,28.61,28.20,131950.72,0.03,0.10,116455.14,101490.42,134514.24,1.14,完美世界,影视音像,24.23,0.01,0.02,get_ma_A_ma30,0.416667,0.25,0.272727,0.222222,0.25


In [84]:
pord_dict['002129']

0.25

In [76]:
stock.query("stock_name=='中环股份'")

,jx_date,var,ma5,ma10,ma20,ma30,jx_counts,jx_names,symbol,t_date,open_price,high,close_price,low,volume,price_change,p_change,v_ma5,v_ma10,v_ma20,turnover,stock_name,industry,pe,bf_close,af_close,sw_class,pord
75246,2019-10-08,0.011018,6.19,6.34,6.20,6.32,4,"6.19,6.34,6.2,6.32",002129,2019-10-08,12.50,12.85,12.64,12.39,754613.38,0.53,4.38,824523.40,1036562.81,979259.42,2.81,中环股份,半导体,61.07,NaN,-0.05,get_ma_A_ma30,0.25
75247,2019-10-09,0.010130,-0.02,-0.01,0.07,0.01,2,"-0.02,-0.01,0.07,0.01",002129,2019-10-09,12.50,12.72,12.66,12.36,510868.12,0.02,0.16,698009.25,952072.86,975822.68,1.90,中环股份,半导体,61.07,NaN,-0.07,get_ma_A_ma30,0.25
75248,2019-10-10,0.007427,0.09,-0.00,0.07,0.07,3,"0.09,-0.0,0.07,0.07",002129,2019-10-10,12.79,13.13,13.02,12.64,839024.06,0.36,2.84,638585.41,895950.62,969930.72,3.13,中环股份,半导体,61.07,NaN,-0.11,get_ma_A_ma30,0.25
75249,2019-10-11,0.004877,0.11,0.00,0.06,0.06,4,"0.11,0.0,0.06,0.06",002129,2019-10-11,13.00,13.01,12.89,12.71,598674.06,-0.13,-1.00,630325.19,861074.88,951538.51,2.23,中环股份,半导体,61.07,NaN,-0.09,get_ma_A_ma30,0.25
75250,2019-10-14,0.007330,0.13,-0.00,0.06,0.06,3,"0.13,-0.0,0.06,0.06",002129,2019-10-14,13.08,13.30,13.04,12.93,752947.25,0.15,1.16,691225.37,804008.89,957539.35,2.81,中环股份,半导体,61.07,NaN,-0.10,get_ma_A_ma30,0.25
75251,2019-10-15,0.007629,0.11,-0.01,0.06,0.04,3,"0.11,-0.01,0.06,0.04",002129,2019-10-15,12.90,13.00,12.65,12.61,577147.50,-0.39,-2.99,655732.20,740127.80,923367.99,2.15,中环股份,半导体,61.07,NaN,-0.09,get_ma_A_ma30,0.25
75252,2019-10-16,0.005932,0.09,-0.02,0.05,0.03,3,"0.09,-0.02,0.05,0.03",002129,2019-10-16,12.69,12.73,12.33,12.25,608876.19,-0.32,-2.53,675333.81,686671.53,899732.63,2.27,中环股份,半导体,61.07,NaN,-0.07,get_ma_A_ma30,0.25
75253,2019-10-17,0.003128,0.05,-0.02,0.03,-0.00,2,"0.05,-0.02,0.03,-0.0",002129,2019-10-17,12.32,12.43,11.96,11.91,705048.94,-0.37,-3.00,648538.79,643562.10,902045.29,2.63,中环股份,半导体,61.07,-0.05,-0.02,get_ma_A_ma30,0.25
75257,2019-11-08,0.007796,0.05,0.01,-0.04,-0.01,2,"0.05,0.01,-0.04,-0.01",002129,2019-11-08,12.01,12.15,12.01,11.87,520988.69,0.08,0.67,517497.13,473564.34,536105.31,1.94,中环股份,半导体,61.07,0.06,-0.07,get_ma_A_ma30,0.25
75258,2019-11-11,0.007769,0.06,0.02,-0.05,-0.01,2,"0.06,0.02,-0.05,-0.01",002129,2019-11-11,11.95,12.21,11.86,11.74,407351.81,-0.15,-1.25,484806.73,450398.45,518825.54,1.52,中环股份,半导体,61.07,0.03,-0.08,get_ma_A_ma30,0.25


In [24]:
stock.sort_values('pord',ascending=False)

,jx_date,var,ma5,ma10,ma20,ma30,jx_counts,jx_names,symbol,t_date,open_price,high,close_price,low,volume,price_change,p_change,v_ma5,v_ma10,v_ma20,turnover,stock_name,industry,pe,bf_close,af_close,sw_class,pord
163345,2020-02-21,0.009549,0.17,0.16,-0.03,0.01,3,"0.17,0.16,-0.03,0.01",002918,2020-02-21,20.00,20.24,19.95,19.80,32583.71,-0.10,-0.50,37074.78,44810.59,46471.89,2.36,蒙娜丽莎,其他建材,25.04,0.03,0.12,get_ma_A_ma30,0.875
163316,2019-10-08,0.010676,7.04,7.14,7.04,6.87,4,"7.04,7.14,7.04,6.87",002918,2019-10-08,14.00,14.29,13.75,13.66,23803.05,-0.22,-1.57,28508.81,36597.86,37615.67,1.76,蒙娜丽莎,其他建材,25.04,NaN,0.13,get_ma_A_ma30,0.875
163344,2020-02-20,0.009824,0.16,0.11,-0.04,0.01,3,"0.16,0.11,-0.04,0.01",002918,2020-02-20,20.16,20.28,20.05,19.61,33354.16,-0.08,-0.40,38782.37,46625.57,45531.17,2.42,蒙娜丽莎,其他建材,25.04,0.02,0.13,get_ma_A_ma30,0.875
163346,2020-02-24,0.010360,0.16,0.19,-0.02,0.00,3,"0.16,0.19,-0.02,0.0",002918,2020-02-24,20.45,20.65,20.00,19.79,56548.01,0.05,0.25,41765.91,44502.13,45426.90,4.10,蒙娜丽莎,其他建材,25.04,0.05,0.20,get_ma_A_ma30,0.875
163347,2020-02-25,0.010930,0.15,0.17,-0.03,-0.00,2,"0.15,0.17,-0.03,-0.0",002918,2020-02-25,19.79,20.38,20.04,19.30,45019.64,0.04,0.20,39335.15,41792.03,45817.82,3.26,蒙娜丽莎,其他建材,25.04,0.06,0.25,get_ma_A_ma30,0.875
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
322894,2019-11-27,0.007101,0.03,0.01,-0.00,-0.00,2,"0.03,0.01,-0.0,-0.0",600721,2019-11-27,5.82,5.94,5.88,5.80,12448.47,0.00,0.00,13428.20,10699.18,11600.67,0.41,ST百花,生物制药,0.00,0.06,0.01,get_ma_A_ma30,0.000
322882,2019-11-11,0.006478,0.01,-0.01,0.00,-0.00,2,"0.01,-0.01,0.0,-0.0",600721,2019-11-11,5.74,5.76,5.56,5.53,12335.00,-0.21,-3.64,13038.45,12702.38,14331.51,0.40,ST百花,生物制药,0.00,0.00,-0.01,get_ma_A_ma30,0.000
322881,2019-11-08,0.005474,0.01,-0.02,0.01,-0.00,2,"0.01,-0.02,0.01,-0.0",600721,2019-11-08,5.76,5.80,5.77,5.72,10556.24,-0.01,-0.17,13216.02,13006.98,14275.49,0.34,ST百花,生物制药,0.00,0.01,-0.04,get_ma_A_ma30,0.000
344955,2020-06-17,0.003558,-0.04,-0.00,0.02,0.02,2,"-0.04,-0.0,0.02,0.02",600897,2020-06-17,17.66,17.76,17.70,17.57,9987.41,0.02,0.11,15772.11,12660.99,10803.38,0.34,厦门空港,机场,40.56,-0.02,0.01,get_ma_A_ma30,0.000


In [77]:
df =  stock.loc[(stock.stock_name=='中环股份') & (stock['var']<=0.015) & (stock.sw_class=='get_ma_A_ma30')&
         (stock.jx_counts>=2)]

In [79]:
date_list = {}
for date in df.jx_date:
    month = date.strftime('%Y%m')
    if month not in date_list:
        date_list[month] = date
date_list = [date_list[i] for i in date_list]

In [80]:
date_list

[Timestamp('2019-10-08 00:00:00'),
 Timestamp('2019-11-08 00:00:00'),
 Timestamp('2019-12-24 00:00:00'),
 Timestamp('2020-03-19 00:00:00'),
 Timestamp('2020-04-24 00:00:00'),
 Timestamp('2020-08-04 00:00:00'),
 Timestamp('2020-09-17 00:00:00'),
 Timestamp('2020-10-09 00:00:00'),
 Timestamp('2020-11-02 00:00:00')]

In [169]:
pa = origin[origin.name=='完美世界'].sort_values('date').set_index('date',drop=False)

In [170]:
pa

,date,open,high,close,low,volume,price_change,p_change,ma5,ma10,ma20,v_ma5,v_ma10,v_ma20,turnover,symbol,name,industry,pe,af_colse_15
date,,,,,,,,,,,,,,,,,,,,
2019-08-01,2019-08-01,26.33,26.56,26.29,26.04,85898.89,0.11,0.42,25.602,24.760,24.518,90333.58,66499.82,64949.43,0.73,002624,完美世界,影视音像,24.23,0.114492
2019-08-02,2019-08-02,25.45,26.69,26.30,25.45,97447.22,0.01,0.04,26.022,25.021,24.553,99373.70,71756.98,66389.96,0.82,002624,完美世界,影视音像,24.23,0.104563
2019-08-05,2019-08-05,26.42,26.99,25.84,25.77,90346.52,-0.46,-1.75,26.182,25.275,24.609,101963.58,77301.45,68137.82,0.76,002624,完美世界,影视音像,24.23,0.089783
2019-08-06,2019-08-06,25.22,26.19,25.68,25.05,105287.03,-0.16,-0.62,26.058,25.464,24.657,95627.52,83903.56,71274.22,0.91,002624,完美世界,影视音像,24.23,0.101246
2019-08-07,2019-08-07,26.20,26.32,25.00,24.98,95323.16,-0.68,-2.65,25.822,25.529,24.702,94860.56,88215.11,72838.14,0.82,002624,完美世界,影视音像,24.23,0.104000
2019-08-08,2019-08-08,25.09,25.54,24.81,24.62,68591.96,-0.19,-0.76,25.526,25.564,24.753,91399.18,90866.38,73665.58,0.59,002624,完美世界,影视音像,24.23,0.112858
2019-08-09,2019-08-09,24.95,25.04,24.24,24.00,99209.42,-0.57,-2.30,25.114,25.568,24.771,91751.62,95562.66,76210.49,0.86,002624,完美世界,影视音像,24.23,0.105611
2019-08-12,2019-08-12,24.65,26.02,25.95,24.53,122288.57,1.71,7.05,25.136,25.659,24.856,98140.03,100051.80,78365.34,1.05,002624,完美世界,影视音像,24.23,0.062813
2019-08-13,2019-08-13,25.93,26.38,26.24,25.56,77901.73,0.29,1.12,25.248,25.653,24.943,92662.97,94145.25,79364.80,0.67,002624,完美世界,影视音像,24.23,0.062881


In [164]:
origin['af_colse_15'] = -origin.groupby('symbol').apply(lambda x: x['close'].diff(-15) / x['close'] ).\
                          reset_index().sort_values('level_1')['close']

In [155]:
origin

,date,open,high,close,low,volume,price_change,p_change,ma5,ma10,ma20,v_ma5,v_ma10,v_ma20,turnover,symbol,name,industry,pe,af_colse_15
0,2019-08-01,14.06,14.19,14.10,13.94,527981.25,-0.03,-0.21,14.224,14.080,13.912,662044.20,692100.19,682379.44,0.31,000001,平安银行,银行,11.59,0.014894
1,2019-08-02,13.77,13.88,13.74,13.66,969926.50,-0.36,-2.55,14.126,14.055,13.903,728968.35,702453.76,693415.97,0.56,000001,平安银行,银行,11.59,0.066230
2,2019-08-05,13.60,13.64,13.35,13.27,893082.38,-0.39,-2.84,13.938,14.005,13.891,764469.59,733533.68,699149.39,0.52,000001,平安银行,银行,11.59,0.067416
3,2019-08-06,13.10,13.46,13.37,13.03,882499.12,0.02,0.15,13.738,13.966,13.880,781642.55,771383.16,718559.54,0.51,000001,平安银行,银行,11.59,0.070307
4,2019-08-07,13.49,13.64,13.54,13.37,793038.94,0.17,1.27,13.620,13.932,13.879,813305.64,793659.43,740300.44,0.46,000001,平安银行,银行,11.59,0.053914
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1141796,2020-11-03,234.36,238.00,234.90,226.10,8666.36,1.85,0.79,238.446,230.262,238.164,15568.83,13920.11,13491.76,5.95,688399,硕世生物,医疗保健,17.68,NaN
1141797,2020-11-04,237.15,244.00,232.37,231.37,8760.70,-2.53,-1.08,238.922,230.469,237.513,13769.45,13740.42,13357.48,6.01,688399,硕世生物,医疗保健,17.68,NaN
1141798,2020-11-05,235.50,236.99,233.02,226.10,8627.55,0.65,0.28,235.864,232.331,236.195,10658.49,13389.28,13055.79,5.92,688399,硕世生物,医疗保健,17.68,NaN
1141799,2020-11-06,234.21,234.97,221.00,218.80,9546.21,-12.02,-5.16,230.868,233.861,234.262,9728.02,13505.92,12686.82,6.55,688399,硕世生物,医疗保健,17.68,NaN


In [213]:
c = pa[pa.date.isin(date_list)].af_colse_15>0.1
len(c) / c.sum()

3.25

In [217]:
c.sum() / len(c)

0.3076923076923077

In [161]:
date_list

[Timestamp('2019-10-08 00:00:00'),
 Timestamp('2019-11-08 00:00:00'),
 Timestamp('2019-12-24 00:00:00'),
 Timestamp('2020-03-19 00:00:00'),
 Timestamp('2020-04-24 00:00:00'),
 Timestamp('2020-08-04 00:00:00'),
 Timestamp('2020-09-17 00:00:00'),
 Timestamp('2020-10-09 00:00:00'),
 Timestamp('2020-11-02 00:00:00')]

In [97]:
(14.31 - 14.10)/14.10

0.014893617021276657